tech :
- langchain
- github
- pincone(vector dbs)
- llm (groq, gemini, mistreal)
- python (3.10)
- flask

GOING TO ROOT DIRECTORY OF PROJECT

In [7]:
print("WORKING ")

WORKING 


In [5]:
%pwd

'c:\\Users\\HP\\Documents\\Q&A_MEDICAL_BOT\\MedicalChatBot\\research'

In [6]:
import os 
os.chdir("../")

In [4]:
%pwd

'c:\\Users\\HP\\Documents\\Q&A_MEDICAL_BOT\\MedicalChatBot'

# STARTING WITH IMPORTANT FUNCTIONS

In [5]:
pdf_path="Data/Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf"
dir_path="Data/"

In [6]:


from langchain.document_loaders import PyPDFLoader,DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [7]:
# load and extract document(tex) from pdf

def load_extract_pdf(path):
    loader=DirectoryLoader(path,glob="*.pdf",loader_cls=PyPDFLoader)
    document=loader.load()
    return document

In [8]:
extracted_data=load_extract_pdf(dir_path)

In [57]:
len(extracted_data)

637

# ANALYSE DATA

In [ ]:
%pip install pandas
import pandas as pd


In [93]:
def analyse_pages(extracted_data):
    data={"character_count":[],"words_len":[],"sentances_len":[],"token_count":[]}
    for i in extracted_data:
        page_data=i.page_content
    
        data["character_count"].append(len(page_data))
        data["words_len"].append(len(page_data.split(" ")))
        data["sentances_len"].append(len(page_data.split(".")))
        data["token_count"].append(len(page_data)/4)

    return data
        
        

In [94]:
df=pd.DataFrame(analyse_pages(extracted_data))
df.describe()

,character_count,words_len,sentances_len,token_count
count,637.000000,637.000000,637.000000,637.000000
mean,4128.039246,560.189953,46.400314,1032.009812
std,786.992492,118.289274,23.092748,196.748123
min,0.000000,1.000000,1.000000,0.000000
25%,3812.000000,502.000000,36.000000,953.000000
50%,4188.000000,563.000000,44.000000,1047.000000
75%,4585.000000,629.000000,55.000000,1146.250000
max,5929.000000,885.000000,443.000000,1482.250000


In [96]:
# convert documents into chunks ! chunk_size=500 and chunk_overlay=30

def document_to_chunks(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500,chunk_overlap=30)
    chunks=text_splitter.split_documents(extracted_data)
    return chunks

In [97]:
chunks=document_to_chunks(extracted_data)

In [ ]:
for i,chunk in enumerate(chunks[:5]):
    print(f"{i} th chunk {chunk}")

In [24]:
print(f"length of chunks is : {len(chunks)}")

length of chunks is : 5876


In [99]:
df=pd.DataFrame(analyse_pages(chunks))
df.describe()

,character_count,words_len,sentances_len,token_count
count,5876.000000,5876.000000,5876.000000,5876.000000
mean,448.913206,61.772634,5.983492,112.228302
std,83.738848,12.420136,5.907419,20.934712
min,38.000000,2.000000,1.000000,9.500000
25%,454.000000,58.000000,4.000000,113.500000
50%,471.000000,64.000000,5.000000,117.750000
75%,487.000000,69.000000,6.000000,121.750000
max,500.000000,88.000000,315.000000,125.000000


# MAKING EMBEDDINGS 

In [10]:
from langchain.embeddings import HuggingFaceBgeEmbeddings

## **🔹 Model Specifications:**
| Specification  | Details  |
|---------------|----------|
| **Model Name** | `all-MiniLM-L6-v2` |
| **Architecture** | MiniLM-L6 (6-layer Transformer) |
| **Embedding Size** | 384-dimensional vector |
| **Training Data** | Diverse text corpora (including Wikipedia, news, question-answer pairs) |
| **Max Token Length** | 256 tokens |
| **Performance** | Small & fast, optimized for efficiency |
| **Use Case** | Text embeddings for semantic search, similarity, retrieval |
| **Framework** | Built using **Hugging Face Transformers** & **Sentence-Transformers** |



In [11]:
def download_huggingface_embedding():
    embeddings=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

## **🔹 Other Models Like This**
There are other **sentence embedding models** similar to `all-MiniLM-L6-v2`, with different trade-offs in terms of **accuracy, speed, and resource consumption**.

| Model | Dim | Speed | Accuracy | Use Case |
|-------|-----|-------|----------|----------|
| **all-MiniLM-L6-v2** | 384 | 🚀 Fast | ⭐⭐⭐ | General-purpose, low-resource apps |
| **all-MiniLM-L12-v2** | 384 | ⏳ Slower | ⭐⭐⭐⭐ | Higher accuracy, still lightweight |
| **all-MPNet-base-v2** | 768 | 🐌 Slow | ⭐⭐⭐⭐⭐ | Best for accuracy, large models |
| **multi-qa-MiniLM-L6-cos-v1** | 384 | 🚀 Fast | ⭐⭐⭐ | Optimized for **Q&A retrieval** |
| **e5-large-v2** | 1024 | 🐌 Slow | ⭐⭐⭐⭐⭐ | High-accuracy embeddings, better for **retrieval** |
| **bge-m3** | 1024 | 🐌 Slow | ⭐⭐⭐⭐⭐ | State-of-the-art, better for **search & RAG** |

🔹 **MiniLM models** → Best for **efficiency and small-scale applications**.  
🔹 **MPNet / E5 / BGE models** → Best for **higher accuracy & retrieval tasks** but require more resources.


In [12]:
embedding_model=download_huggingface_embedding()

C:\Users\HP\AppData\Local\Temp\ipykernel_13116\2672646417.py:2: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
c:\Users\HP\anaconda3\envs\medibot1\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_embed=embedding_model.embed_query("Hello world")
print(f'length of the embedding vetor {len(query_embed)}')

length of the embedding vetor 384


# MAKING PINECONE DATABASE

In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY=os.environ.get("PINECONE_API_KEY")

In [16]:
! pip uninstall pinecone-plugin-inference

In [100]:
from pinecone.grpc import PineconeGRPC as Pinecone
from pinecone import ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medibotembed"

In [ ]:

pc.create_index(

    name=index_name,
    dimension=384, 
    metric="cosine", 
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [18]:
os.environ["PINECONE_API_KEY"]=PINECONE_API_KEY

In [ ]:
from langchain_pinecone import PineconeVectorStore

docsearch=PineconeVectorStore.from_documents(
    index_name=index_name,
    documents=chunks,
    embedding=embedding_model
)




✅ **What happens in this step?**
1. **Embeds the documents** (`chunks`) using the `embedding_model`.
2. **Stores the embeddings** in the Pinecone index (`index_name`).
3. **Creates a searchable vector database**, enabling similarity-based searches.




In [19]:
from langchain_pinecone import PineconeVectorStore

docretrival=PineconeVectorStore.from_existing_index(
    index_name=index_name,
    embedding=embedding_model
)


In [20]:
docretrival.similarity_search("what is acne?",k=5)

[Document(id='a533e936-3c3a-44d1-abd8-ced0e1f64127', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='69294992-466c-4694-a8df-89293bdbb8e0', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nate

In [29]:
reteriver=docretrival.as_retriever(search_type="similarity",search_kwargs={"k":5})
reteriver.invoke("what is acne?")

[Document(id='a533e936-3c3a-44d1-abd8-ced0e1f64127', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 39.0, 'page_label': '40', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='69294992-466c-4694-a8df-89293bdbb8e0', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 38.0, 'page_label': '39', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nate

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-002",temperature=0.6,top_k=40,top_p=0.9)
llm.invoke("Write me a ballad about LangChain")

AIMessage(content="The coder toiled, in screen's pale light,\nWith APIs a-scattered, day and night.\nA thousand calls, a million lines,\nTo weave a web, of complex designs.\nHe dreamed of chains, not forged of steel,\nBut links of thought, that would reveal\nThe secrets held in data's store,\nAnd answer questions, evermore.\n\nThen from the depths, a whisper came,\nA framework born, LangChain by name.\nIt promised ease, a simpler way,\nTo build the chains, and make them play.\nNo more the toil, the endless grind,\nTo link each piece, and leave behind\nThe tangled mess, the cryptic code,\nLangChain's embrace, a lighter load.\n\nWith LLMs grand, and agents keen,\nIt built the bridges, in between.\nFrom prompt to query, swift and true,\nThe data flowed, the answers grew.\nThe coder smiled, his work complete,\nA chain of thought, both strong and neat.\nHe whispered thanks, with grateful heart,\nFor LangChain's magic, a brand new start.\n\nBut whispers turned to shouts of glee,\nAs others j

In [26]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

In [27]:
system_prompt = """
You are an intelligent and helpful assistant designed for answering questions.
Use the provided retrieved context to generate a clear and concise response.
- If the context is relevant, answer in a simple and easy-to-understand manner.
- If the context does not contain the answer or is unrelated to the query, respond with: "I don't know."

Context:
{context}
"""

prompt=ChatPromptTemplate.from_messages(
    [
        ("system",system_prompt),
        ("human","{input}")
    ]
)

In [30]:
q_a_document_chain=create_stuff_documents_chain(llm,prompt)
rag_chain=create_retrieval_chain(reteriver,q_a_document_chain)

| Step | Process |
|------|---------|
| **1. `create_stuff_documents_chain`** | Formats retrieved docs and passes them to LLM |
| **2. `create_retrieval_chain`** | Retrieves relevant documents and feeds them into `q_a_document_chain` |
| **3. Query RAG Chain** | User asks a question → Retrieval → Formatting → LLM generates response |

✅ **Use `q_a_document_chain`** when you already have retrieved documents.  
✅ **Use `rag_chain`** when you need to retrieve and process documents dynamically.  


In [31]:
response=rag_chain.invoke({"input":"what is acne?"})
print(response["answer"])

Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when pores become clogged with oil, dead skin cells, and bacteria.


In [52]:
# making custome chain 

def custom_rag(query):
    retrived_document=reteriver.invoke(query)
    print(f"retrived similiar documents : {retrived_document}")
    context_text = "\n".join([doc.page_content for doc in retrived_document])
    formatted_prompt=prompt.format(context=context_text,input=query)
    response=llm.invoke(formatted_prompt)
    return response.content


In [55]:
print(custom_rag("who is jainam?"))

retrived similiar documents : [Document(id='d502b057-8bf6-40b1-ac92-2d73138ed75b', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddate': '2004-12-18T16:15:31-06:00', 'page': 443.0, 'page_label': '444', 'producer': 'PDFlib+PDI 5.0.0 (SunOS)', 'source': 'Data\\Gale Encyclopedia of Medicine Vol. 1 (A-B).pdf', 'total_pages': 637.0}, page_content='remedies, and conducts research and documentation of\nits healing techniques.\nKey ideas\nTo understand Ayurvedic treatment, it is necessary to\nhave an idea how the Ayurvedic system views the body.\nThe basic life force in the body is prana, which is also\nfound in the elements and is similar to the Chinese\nnotion of chi. As Swami Vishnudevananda, a yogi and\nexpert, put it, “Prana is in the air, but is not the oxygen,\nnor any of its chemical constituents. It is in food, water,'), Document(id='1e9e4ecc-8cb8-470b-ac52-5cc658f8dea8', metadata={'creationdate': '2004-12-18T17:00:02-05:00', 'creator': 'PyPDF', 'moddat

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI
from dotenv import load_dotenv
load_dotenv()
llm = ChatGoogleGenerativeAI(model="gemini-1.5-pro-002",temperature=0.6,top_k=40,top_p=0.9)
llm.invoke("Write me a ballad about LangChain")